In [ ]:
if __name__ == "__main__":
    
    torch.manual_seed(0)
    # Initialization
    N_MARCHING_CUBE = 64
    lr= 8e-3
    l2reg= True
    regl2 = 1e-3
    decreased_by = 1.5
    adjust_lr_every = 50
    

    
    # pick initialization and samples
    # Load collection of all latent codes
    all_codes_path = os.path.join(
        args.experiment_directory,
        ws.latent_codes_subdir,
        'latest.pth')
    all_codes = torch.load(all_codes_path)['latent_codes']['weight']
    ## sphere
    source_id = 999 # zywvjkvz2492e6xpq4hd1jzy2r9lht        # This will be the source shape (ie starting point)
    latent = all_codes[source_id].unsqueeze(0).detach().cuda()   #Add .cuda() if you want to run on GPU
    latent.requires_grad = True

    ## torus
    target_id = 2 # 0bucd9ryckhaqtqvbiagilujeqzek4          # This is be the target shape (ie objective)
    latent_target = all_codes[target_id].unsqueeze(0).detach().cuda()   #Add .cuda() if you want to run on GPU

     
    
    specs_filename = os.path.join(args.experiment_directory, "specs.json")
    specs = json.load(open(specs_filename))
    arch = __import__("networks." + specs["NetworkArch"], fromlist=["Decoder"])
    latent_size = specs["CodeLength"]
    # Load decoder: this is our black box function
    decoder = arch.Decoder(latent_size, **specs["NetworkSpecs"])
    decoder = torch.nn.DataParallel(decoder)
    saved_model_state = torch.load(
        os.path.join(
            args.experiment_directory, ws.model_params_subdir, args.checkpoint + ".pth"
        ),
        map_location=torch.device('cpu') # Remove this if you want to run on GPU
    )
    decoder.load_state_dict(saved_model_state["model_state_dict"])
    # Optionally: put decoder on GPU
    decoder = decoder.module.cuda()


    optimizer = torch.optim.Adam([latent], lr=lr)

    losses2 = []
    lambdas = []

    # Use Adam optimizer, with source as starting point, and a loss defined on meshes
    # latent is the input of our function
    print("Starting optimization:")
    for e in range(int(args.iterations)):

        if latent.grad is not None:
            latent.grad.detach_()
            latent.grad.zero_()

        verts, faces = deep_sdf.mesh.create_mesh_optim(decoder, latent, N=N_MARCHING_CUBE, max_batch=int(2 ** 18))

        
        # subsample vertices for gradients computations
        verts = verts[torch.randperm(verts.shape[0])]
        verts = verts[0:20000, :]
        # forward pass within loss layer
        xyz_upstream = torch.tensor(verts.astype(float), requires_grad = True, dtype=torch.float32, device=torch.device('cuda:0')) # For GPU,
        # Get a point cloud sampling of the target shape
        verts_target_sample = verts_target[torch.randperm(verts_target.shape[0])]
        verts_target_sample = verts_target_sample[0:20000, :]
        xyz_target = torch.tensor(verts_target_sample.astype(float), requires_grad = False, dtype=torch.float32, device=torch.device('cuda:0')) # For GPU, add: , device=torch.device('cuda:0'))
        # At this point we have 2 outputs for decoder: the target xyz_target, and the current value xyz_upstream
        # The following lines compute a loss and backpropagate
        # compute loss function: Chamfer between current guess (xyz_upstream) and objective (xyz_target)
        loss = chamfer_distance(xyz_upstream, xyz_target)
        print("Loss at iter", e, ":", loss.item(), ", latent norm: ", torch.norm(latent))
        
        
        losses2.append(loss.detach().cpu().numpy())                                  ## Loss value
        lambdas.append(torch.norm(latent_target-latent).detach().cpu().numpy())     ## Distance in the domain
        decoder.eval()
        loss.backward()
        dL_dx_i = xyz_upstream.grad
        
        # use vertices to compute full backward pass
        xyz = torch.tensor(verts.astype(float), requires_grad = True, dtype=torch.float32, device=torch.device('cuda:0'))# # For GPU,
        latent_inputs = latent.expand(xyz.shape[0], -1)
        inputs = torch.cat([latent_inputs, xyz], 1).cuda()      #Add .cuda() if you want to run on GPU
        #first compute normals
        pred_sdf = decoder(inputs)
        loss_normals = torch.sum(pred_sdf)
        loss_normals.backward(retain_graph = True)
        normals = xyz.grad/torch.norm(xyz.grad, 2, 1).unsqueeze(-1)
        
        # now assemble inflow derivative
        latent.grad.detach_()
        latent.grad.zero_()
        dL_ds_i_fast = -torch.matmul(dL_dx_i.unsqueeze(1), normals.unsqueeze(-1)).squeeze(-1)
        loss_backward = torch.sum(dL_ds_i_fast * pred_sdf)
        if e % 20 == 0 and e > 0:
            regl2 = regl2/2
        if l2reg:
            loss_backward += regl2* torch.mean(latent.pow(2))
        # Backpropagate
        loss_backward.backward()

        
       # print("time to backward:", end-start)
        # update latent
        # Explicit gradient is accessible via latent.grad
        
        adjust_learning_rate(lr, optimizer, e, decreased_by, adjust_lr_every)
        optimizer.step()


        # 1. objective function value
        print("loss backward:", loss_backward.item())
        # 2. its derivative function value on current arguments 
        print("\n")
        #print(latent.grad)
        #print("shape of verts_target, faces_target: ", verts_target.shape, faces_target.shape, xyz_target.shape)
        #raise Exception("Stop");
        